## Supervisely SDK example

This notebook is an example of how to use *sly_sdk_utils.py* file.

In [13]:
# import libraries

import supervisely as sly
from sly_sdk_utils import change_cls_annotations, remove_pt_ann
import cv2


ModuleNotFoundError: No module named 'sly_sdk_utils'

### Connect to Supervise.ly API

In [ ]:
# Supervise.ly token
key = "..."
api = sly.Api(server_address="https://app.supervise.ly", token=key)

### Download Supervise.ly project

Download the project which has to be modified thanks to Supervise.ly Python SDK.

In [ ]:
project_id = 257879

save_directory = "legio_evry_SM"
os.makedirs(save_directory)
# Download Project
sly.Project.download(api, project_id, save_directory)
project_fs = sly.Project(save_directory, sly.OpenMode.READ)
dataset_path = os.path.join(os.getcwd(), save_directory, "ds0")
ds = sly.Dataset(dataset_path, sly.OpenMode.READ)

# Get object classes statistics
ds.get_classes_stats()

### Create new Supervise.ly project

Create a new Supervise.ly project which will contain the modified data of downloaded project.

In [ ]:
project_path = "/content/Legio_Evry_SM_rework"
new_project = sly.Project(project_path, sly.OpenMode.CREATE)

# Set meta of previous project
new_project.set_meta(project_fs.meta.clone())

# Add new dataset
new_ds_name = "ds0"
new_project.create_dataset(new_ds_name)
new_ds = sly.Dataset(os.path.join(project_path, new_ds_name), sly.OpenMode.CREATE)

### Fill new project with modified data

Fill new project with modified data. In this case, some object classes of labels are replaced by others and labels with area which equals to 1 are deleted.

In [ ]:
dataset_path = "/content/legio_evry_SM/ds0"
ds = sly.Dataset(dataset_path, sly.OpenMode.READ)

for item_name, image_path, ann_path in ds.items():
    print(f"Item '{item_name}': image='{image_path}', ann='{ann_path}'")

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    annotation = ds.get_ann(item_name, project_fs.meta)
    
    # modify data
    # Replace "Added" boject class by "Model" object class for all labels
    new_ann = change_cls_annotations(annotation=annotation, input_classes=["Added"], 
                                   output_classes=["Model"], meta=project_fs.meta)
    # Remove labels which have an area equal to 1
    new_ann = remove_pt_ann(annotation=new_ann)

    # Add item on new project
    new_ds.add_item_np(item_name=item_name, img=image, ann=new_ann, img_info=None)

### Upload Supervise.ly dataset

Upload new project into Supervisely platform.

In [ ]:
workspace_id = 43377
project_name = "Legio_Evry_SM_rework"

project_id, project_name = sly.Project.upload(
    new_project.directory,
    api,
    workspace_id=workspace_id,
    project_name=project_name
)